# 00_Gerating_artificial_signal_peptides

## 0_Introduction:
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda:
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

In [ ]:
import numpy as np
import pandas as pd
import random
import os

In [ ]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str


In [ ]:
def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
    
    
    return sequence        

In [ ]:
def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


In [ ]:
def split_peptides_sequences(df_signalPP:pd.DataFrame): 
    '''Split each AA for each position'''
    peptides_split = []
    for k,v in df_signalPP.iterrows(): 
        sequence = []
        for seq in v['sequence']: 
            sequence.append(seq)
        peptides_split.append(sequence)
    
    # make a dataframe
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides

## 1_AA_probability_matrix

Lets import our df_pwn dataframe that was made in a previous notebook and use the position specific scoring matrix for the generation of novel signal peptides

In [ ]:
from google.colab import drive 
drive.mount('/content/home')

Mounted at /content/home


In [ ]:
df_pwn =  pd.read_csv('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/02_All_signal_peptides/df_pwn_68_positions.csv')
df_pwn

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.069877,0.001889,0.002833,0.001889,0.032106,0.021719,0.064212,0.037771,0.208687,0.084986,...,0.022663,0.028329,0.058546,0.193579,0.033994,0.018886,0.052880,0.014164,0.028329,0.000000
2,0.044381,0.007554,0.006610,0.007554,0.152975,0.047214,0.019830,0.050047,0.018886,0.190746,...,0.020774,0.052880,0.020774,0.053824,0.131256,0.053824,0.060434,0.025496,0.022663,0.000000
3,0.057602,0.001889,0.004721,0.004721,0.089707,0.019830,0.016053,0.064212,0.058546,0.175637,...,0.033994,0.041549,0.043437,0.056657,0.163362,0.100094,0.021719,0.010387,0.026440,0.000000
4,0.072710,0.004721,0.004721,0.007554,0.054769,0.032106,0.019830,0.077432,0.053824,0.155807,...,0.041549,0.040604,0.036827,0.045326,0.138810,0.096317,0.043437,0.023607,0.032106,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.001889,0.000944,0.000000,0.000000,0.000000,0.997167
59,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.998111
60,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.998111
61,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [ ]:
amino_acids = list(df_pwn.columns.values)

In [ ]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())

## 2_Load_best_model_from_AutoML

In [ ]:
%%capture 
!pip install h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpf2z7u0ez
  JVM stdout: /tmp/tmpf2z7u0ez/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpf2z7u0ez/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_unknownUser_t3qxta
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
best_model = h2o.load_model("/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/06_H2O_AutoML/06.1_Best_models/0803_m30_filtered_DeepLearning_grid_1_AutoML_2_20230308_164525_model_14")
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_2_20230308_164525_model_14


Status of Neuron Layers: predicting abundance, regression, gaussian distribution, Quadratic loss, 59,701 weights/biases, 708.6 KB, 279,837 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms          mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  ------------------  --------------------  -----------------------
    1        595      Input             15.0
    2        100      RectifierDropout  40.0       0.0   0.0   0.08383776184387944    0.22281700372695923    0.0         -0.024995468157608204  0.0911882221698761  0.2398722263720472    0.14759206771850586
    3        1        Linear                       0.0   0.0   0.0015805453458415286  0.0027047647163271904  0.0         0.06839000282227062    0.147660493850708   -0.09775900897466194  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0006588389682246397
RMSE: 0.02566785866067989
MAE: 0.0024433722974568006
RMSLE: 0.018117105551137377
Mean Residual Deviance: 0.0006588389682246397

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0014665389530156953
RMSE: 0.038295416866978946
MAE: 0.00556754191609704
RMSLE: 0.02895905723013147
Mean Residual Deviance: 0.0014665389530156953

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.0056371   0.00324635  0.0118159     0.010583      0.00585244    0.00695106    0.004379      0.00414332    0.003345      0.00427296    0.00254726    0.00248104
mean_residual_deviance  0.00145843  0.00296455  0.00974658    0.00172341    0.00117409    0.000772287   0.000491289   0.000395702   0.000149314   3.01591e-05   6.2273e-05    3.9206e-05
mse                     0.00145843  0.00296455  0.00974658    0.00172341    0.00117409    0.000772287   0.000491289   0.000395702   0.000149314   3.01591e-05   6.2273e-05    3.9206e-05
r2                      -0.222726   0.613278    0.00516749    -0.00454644   -0.01919      -0.0130522    -0.0227253    -0.018934     -0.013826     -1.96566      -0.101024     -0.0734686
residual_deviance       0.00145843  0.00296455  0.00974658    0.00172341    0.00117409    0.000772287   0.000491289   0.000395702   0.000149314   3.01591e-05   6.2273e-05    3.9206e-05
rmse                    0.0276215   0.0277985   0.0987247     0.041514      0.0342649     0.0277901     0.022165      0.0198923     0.0122194     0.00549173    0.00789132    0.00626147
rmsle                   0.0226405   0.0189134   0.0682184     0.0348133     0.0296352     0.0246152     0.0201131     0.0181916     0.0115827     0.0054735     0.00766329    0.0060985

Scoring History: 
    timestamp            duration           training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  -----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-03-08 17:43:13  0.000 sec                            0         0             0          nan              nan                  nan             nan
    2023-03-08 17:43:13  57 min 39.857 sec  8597 obs/sec      3         1             3009       0.0377321        0.00142371           0.00899153      0.0318429
 

## 3_Signal_peptide_predictor_algorithm

In [ ]:
def signal_peptide_predictor(list_of_probabilities, amino_acids, n_peptides,  number_of_iterations:int)-> pd.DataFrame:
    '''Predicts best signal peptides from a number of iterations'''

    data = pd.DataFrame()
    for i in range(0,number_of_iterations):
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len = 22 )
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        # make the df into categorical values
        for column in df_test.columns:
            if column != 'MM_N_peptide_abundance':
                df_test[column] = df_test[column].asfactor()

        #predict
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        if len(data) == 0: 
            data = new_TO_NATURE_peptides.copy()
        else: 
            # concat to precious predictions
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending = False)
            data = data[0:1000]   

        if i % 1000 == 0:
            print(f"Iteration {i}")
            
    return data

In [ ]:
%%time
lets_predict_signal_peptides = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids,n_peptides = 1000,  
                                                        number_of_iterations = 2000)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Iteration 0


/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column '23': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column '22': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column '25': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column '10' has levels not trained on: ["D"]
  warnings.warn(w)
/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column '26': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column '24': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.9/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column '21' has levels not trained on: ["M"]
  warnings.warn(w)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [ ]:
lets_predict_signal_peptides

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,predictions
47,M,L,R,V,I,A,I,A,T,L,...,Q,A,A,A,A,-,-,-,-,0.202115
986,M,I,S,F,N,A,R,N,F,L,...,Q,A,Y,G,P,N,V,-,-,0.188303
286,M,W,F,V,S,I,H,F,F,A,...,G,I,T,V,A,-,-,-,-,0.174510
35,M,L,W,F,S,A,W,L,Y,A,...,L,I,L,L,A,-,-,-,-,0.172323
42,M,V,V,F,T,W,R,F,L,L,...,V,A,A,L,P,L,Y,-,-,0.165923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,M,H,I,V,S,F,I,L,A,L,...,V,P,A,L,P,F,A,-,-,0.068237
212,M,Q,I,F,S,A,I,L,F,L,...,N,V,T,P,-,-,-,-,-,0.068229
4,M,K,M,Y,H,L,I,T,M,A,...,V,V,L,A,-,-,-,-,-,0.068216
596,M,L,R,I,S,A,L,L,L,G,...,G,A,S,L,A,S,S,V,-,0.068216


In [ ]:
lets_predict_signal_peptides.to_excel('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/09-AutoML/0803_m30_lets_predict_signal_peptides.xlsx', index=False)
